<a href="https://colab.research.google.com/github/Fraol123/Telegrambot/blob/main/Canal%2B_Customer_support_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install python-telegram-bot==13.7 --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.1/490.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.2
    Uninstalling cachetools-5.3.2:
      Successfully uninstalled cachetools-5.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup, ParseMode
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler, MessageHandler, Filters, CallbackContext
from telegram.error import BadRequest

# Simulated "database" for demonstration purposes
users_database = {
    '12345': {'name': 'Ato Ekle', 'balance': 'ETB 794.0 (USD 14.69)', 'package': 'CANAL+ Meda', 'status': 'Disconnected', 'due_date': '06/02/2024'},
}

# A set to keep track of logged-in users (by user_id)
logged_in_users = {}

def start(update: Update, context: CallbackContext) -> None:
    buttons = [
        [InlineKeyboardButton("Sign In", callback_data='sign_in'),
         InlineKeyboardButton("Activate New CANAL+", callback_data='activate')]
    ]
    context.bot.send_photo(
        chat_id=update.effective_chat.id,
        photo='https://scontent.fadd2-1.fna.fbcdn.net/v/t39.30808-6/304995089_488283769975018_8483364646892795396_n.jpg?_nc_cat=110&ccb=1-7&_nc_sid=efb6e6&_nc_ohc=rkc16eLAGV0AX_M24O-&_nc_ht=scontent.fadd2-1.fna&oh=00_AfAcbhW2oeSSCA4KUvE1EbltygvQr8fpgEOjbzrHzaSjxA&oe=65C80F29',  # Replace with the actual URL of your welcome image
        caption="Welcome to CANAL+.🙂\nLet's get started.",
        reply_markup=InlineKeyboardMarkup(buttons)
    )

def button_handler(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    if query is None:
        return

    query.answer()
    data = query.data
    user_id = update.effective_user.id

    if data == 'sign_in' or data == 'activate':
        message = "Please reply with your CANAL+ ID." if data == 'sign_in' else "Your new CANAL+ account has been activated."
        context.bot.send_message(chat_id=query.message.chat_id, text=message)
    elif data in ('check_balance', 'fix_errors', 'pay', 'change_package'):
        handle_actions(data, query, user_id, context)
    elif data == 'get_updates':
        context.bot.send_message(
            chat_id=query.message.chat_id,
            text="Click here to join our [CANAL+ Updates Channel](https://t.me/canalplusbusiness).",  # Update with your channel's URL
            parse_mode=ParseMode.MARKDOWN,
            disable_web_page_preview=True
        )
    elif data == 'back_to_menu' or data == 'logout':
        if data == 'logout' and user_id in logged_in_users:
            del logged_in_users[user_id]
            context.bot.send_message(chat_id=query.message.chat_id, text="You have been logged out of CANAL+.")
        show_main_menu(query.message.chat_id, context, "What would you like to do next?")

def handle_actions(data, query, user_id, context):
    text_responses = {
        'check_balance': "Account Details:\nBalance: ETB 794.0 (USD 14.69)\nStatus: Disconnected\nDue Date: 06/02/2024",
        'fix_errors': "All errors on your decoder have been fixed.",
        'pay': "Your payment has been processed successfully.",
        'change_package': "Your CANAL+ package has been changed to the new selection."
    }
    context.bot.send_message(chat_id=query.message.chat_id, text=text_responses[data], reply_markup=back_to_menu_keyboard())

def message_handler(update: Update, context: CallbackContext) -> None:
    user_id = update.effective_user.id
    text = update.message.text.strip()

    if text.isdigit() and text in users_database and user_id not in logged_in_users:
        logged_in_users[user_id] = text
        show_main_menu(update.message.chat_id, context, f"Hi {users_database[text]['name']}! What would you like to do?")
    else:
        update.message.reply_text("We couldn't match your details. Please try again or contact support.")

def show_main_menu(chat_id: int, context: CallbackContext, message: str):
    buttons = [
        [InlineKeyboardButton("Check your Balance", callback_data='check_balance')],
        [InlineKeyboardButton("Fix Errors on Decoder", callback_data='fix_errors')],
        [InlineKeyboardButton("Pay / Top-Up", callback_data='pay')],
        [InlineKeyboardButton("Change Package", callback_data='change_package')],
        [InlineKeyboardButton("Get Daily Updates", callback_data='get_updates')],
        [InlineKeyboardButton("Log Out", callback_data='logout')],
        [InlineKeyboardButton("Back to Menu", callback_data='back_to_menu')]
    ]
    context.bot.send_message(chat_id=chat_id, text=message, reply_markup=InlineKeyboardMarkup(buttons))

def back_to_menu_keyboard():
    return InlineKeyboardMarkup([[InlineKeyboardButton("Back to Menu", callback_data='back_to_menu')]])

def main() -> None:
    updater = Updater("6951520668:AAGyiZoBNKD_8Losz3uthRAwExPak_rwyDA", use_context=True)  # Replace with your bot's token

    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CallbackQueryHandler(button_handler))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, message_handler))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()
